In [1]:
'''
试图模仿https://mne.tools/stable/auto_tutorials/stats-sensor-space/20_erp_stats.html

进行全部 channel  和 时间点  的t-test

基本单元：某个时间点，某个channel，  所有被试数据是否差异显著。

'''
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

import mne
from mne.channels import find_ch_adjacency, make_1020_channel_selections
from mne.stats import spatio_temporal_cluster_test

np.random.seed(0)

In [2]:
##数据读入 读入的是 clear过的epochs

sub_ids = [

    "pre1001","pre1003","pre1004","pre1005","pre1006",

]

data_path = 'E:\实验\阅读困难\数据处理\pre1nanshi\data\\7evoked_oddball\\'
type_stim = ['std_BA','odd_TA','std_ta','odd_ba','std_t1ji','odd_t4ji','std_t4ji','odd_t1ji']
list_evoked_std_BA = list()
list_evoked_odd_TA = list()
list_evoked_std_ta = list()
list_evoked_odd_ba = list()
# sub_id = sub_ids[1]
for sub_id in sub_ids:
    num_type = 5
    fname = data_path + sub_id + '_'+ 'std_BA' +'-ave.fif'
    evoked1 = mne.read_evokeds(fname)[0]
    list_evoked_std_BA.append(evoked1)

    num_type = 6
    fname = data_path + sub_id + '_'+'odd_TA'+'-ave.fif'
    evoked2 = mne.read_evokeds(fname)[0]
    list_evoked_odd_TA.append(evoked2)

    num_type = 7
    fname = data_path + sub_id + '_'+'std_ta'+'-ave.fif'
    evoked3 = mne.read_evokeds(fname)[0]
    list_evoked_odd_TA.append(evoked3)

    num_type = 8
    fname = data_path + sub_id + '_'+'odd_ba'+'-ave.fif'
    evoked4 = mne.read_evokeds(fname)[0]
    list_evoked_odd_TA.append(evoked4)

list_evoked_std = list_evoked_std_BA+list_evoked_std_ta
list_evoked_odd = list_evoked_odd_TA+list_evoked_odd_ba

grand_average_std = mne.grand_average(list_evoked_std)
grand_average_odd = mne.grand_average(list_evoked_odd)
grand_average_all = list()
grand_average_all = [grand_average_std,grand_average_odd]


adjacency, _ = find_ch_adjacency(list_evoked_odd_TA[0].info, "eeg")
#造出evoked的数据 被试个数*63*501

evokeddata_T1S = np.zeros([len(list_evoked_std_BA),62, 1501], dtype=np.float32)
evokeddata_T2S = np.zeros([len(list_evoked_std_BA),62, 1501], dtype=np.float32)
for i_evoked in range(1,len(sub_ids)):
    evokeddata_T1S[i_evoked,:,:] = list_evoked_std_BA[i_evoked].data
    evokeddata_T2S[i_evoked,:,:] = list_evoked_odd_TA[i_evoked].data
#交换位置
X = [evokeddata_T1S.transpose(0, 2, 1),
     evokeddata_T2S.transpose(0, 2, 1)]

Reading E:\实验\阅读困难\数据处理\pre1nanshi\data\7evoked_oddball\pre1001_std_BA-ave.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms (1)
        0 CTF compensation matrices available
        nave = 337 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.5, 0] sec)
Reading E:\实验\阅读困难\数据处理\pre1nanshi\data\7evoked_oddball\pre1001_odd_TA-ave.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms (2)
        0 CTF compensation matrices available
        nave = 63 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.5, 0] sec)
Reading E:\实验\阅读困难\数据处理\pre1nanshi\data\7evoked_oddball\pre1001_std_ta-ave.fif ...
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms (3)
        0 CTF compensation matrices available
     

In [3]:
tfce = dict(start=.0, step=.01)

# Calculate statistical thresholds
t_obs, clusters, cluster_pv, h0 = spatio_temporal_cluster_test(
    X, tfce, adjacency=adjacency,
    n_permutations=10)  # a more standard number would be 1000+

#关键的就是找到 差异显著的点，后面的section仅仅是作图的问题了。

significant_points = cluster_pv.reshape(t_obs.shape).T < .3
print(str(significant_points.sum()) + " points selected by TFCE ...")

stat_fun(H1): min=-0.000002 max=24.321096
Running initial clustering
Using 2433 thresholds from 0.00 to 24.32 for TFCE computation (h_power=2.00, e_power=0.50)
Found 93062 clusters
Permuting 9 times...


  0%|          |  : 0/9 [00:00<?,       ?it/s]

Computing cluster p-values
Done.
0 points selected by TFCE ...


In [4]:
evoked = mne.combine_evoked([evoked1, evoked2],
                            weights=[1, -1])  # calculate difference wave
time_unit = dict(time_unit="s")
evoked.plot_joint(title="Long vs. short words", ts_args=time_unit,
                  topomap_args=time_unit)  # show difference wave

# Create ROIs by checking channel labels
selections = make_1020_channel_selections(evoked.info)

# Visualize the results
fig, axes = plt.subplots(nrows=3, figsize=(8, 8))
axes = {sel: ax for sel, ax in zip(selections, axes.ravel())}
evoked.plot_image(axes=axes, group_by=selections, colorbar=False, show=False,
                  mask=significant_points, show_names="all", titles=None,
                  **time_unit)
plt.colorbar(axes["Left"].images[-1], ax=list(axes.values()), shrink=.3,
             label="µV")

plt.show()

No projector specified for this dataset. Please consider the method self.add_proj.
